## Integrantes

Nombre 1: Abraham de la Cruz Escalante González

Nombre 2: Marcos Alonso Kú Maás

# Importando librerías

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

np.random.seed(80)

# Obteniendo datos

In [2]:
df_train_test = pd.read_csv('german_train_test.txt', delim_whitespace = True, header = None) #, names = columnas)
df_train_test.head(2)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1,6,4,12,5,5,3,4,1,67,...,0,0,1,0,0,1,0,0,1,1
1,2,48,2,60,1,3,2,2,1,22,...,0,0,1,0,0,1,0,0,1,2


In [3]:
df_validation = pd.read_csv('german_validation.txt', delim_whitespace = True, header = None) #, names = columnas)
df_validation.head(2)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1,16,4,26,1,5,3,4,2,43,...,1,0,0,0,1,0,0,0,1,2
1,4,20,4,35,5,2,1,4,1,44,...,1,0,1,0,0,1,0,0,1,1


## Verificando que no exitan valores nulos

In [4]:
df_train_test.isnull().any().any()

False

In [5]:
df_validation.isnull().any().any()

False

## Creando X, y, X_validation, y_validation

In [6]:
# Creando X
X= df_train_test.drop(24, axis = 1)

# Creando y
y = df_train_test[24]

# Creando X_validation
X_validation= df_validation.drop(24, axis = 1)

# Creando y_validation
y_validation = df_validation[24]

## Dividiendo datos

In [7]:
# Dividiendo datos en datos de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

In [8]:
# Revisando  el tamaño de cada muestra
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((720, 24), (180, 24), (720,), (180,))

# Pipeline

In [9]:
# Modulos importantes
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [10]:
# Mostrando columnas
df_train_test.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24],
           dtype='int64')

In [11]:
# Crear la lista de  viriables numericas  numeric_list 
numeric_list = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

In [12]:
# Crear el pipeline de preprocesamiento para datos numéricos  
numeric_transformer = Pipeline(steps = [('scaler', StandardScaler())])

In [13]:
# Definir los transformadores
transformer = ColumnTransformer([('num', numeric_transformer, numeric_list)])

# Modulos

## Random Forest

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [15]:
# Asignamos nuestro modelo a una variable
forest_clf = RandomForestClassifier(random_state=42)

In [16]:
forest_clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [17]:
# Definiendo el Pipeline
model_forest_clf = Pipeline(steps=[
    ('transformer', transformer),
    ('forest_clf', forest_clf)
])

In [18]:
# Ajustar el modelo
model_forest_clf.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16,
                                                   17, 18, 19, 20, 21, 22,
                                                   23])])),
                ('forest_clf', RandomForestClassifier(random_state=42))])

In [19]:
model_forest_clf.score(X_test, y_test)

0.7833333333333333

In [20]:
max_depth = [2, 11, 15]
min_samples_leaf = [3, 8, 10]
n_estimators = [8, 24, 36]
max_features = ['log2', 'sqrt']

params = {'forest_clf__n_estimators': n_estimators,
          'forest_clf__max_features': max_features,
          'forest_clf__max_depth': max_depth,
          'forest_clf__min_samples_leaf': min_samples_leaf
         }

In [21]:
#Definicion del grid de busqueda
grid_RF = GridSearchCV(model_forest_clf, param_grid=params,
                   cv=10, scoring='neg_mean_squared_error',
                   n_jobs=-1, verbose=2)

In [22]:
# Ajuste del modelo usando todas las posibles combinaciones de parametros 
grid_RF.fit(X, y)

Fitting 10 folds for each of 54 candidates, totalling 540 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4, 5,
                                                                          6, 7,
                                                                          8, 9,
                                                                          10,
                                                                          11,
                                                                          12,
                                                                          13,
                                                                          14,
                                                                          15,
                                                                          16,
                                                                          17,
                                                                          18,
                                                                          19,
                                                                          20,
                                                                          21,
                                                                          22,
                                                                          23])])),
                                       ('forest_clf',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'forest_clf__max_depth': [2, 11, 15],
                         'forest_clf__max_features': ['log2', 'sqrt'],
                         'forest_clf__min_samples_leaf': [3, 8, 10],
                         'forest_clf__n_estimators': [8, 24, 36]},
             scoring='neg_mean_squared_error', verbose=2)

In [23]:
# Mostrar la mejor combinación
grid_RF.best_params_

{'forest_clf__max_depth': 11,
 'forest_clf__max_features': 'log2',
 'forest_clf__min_samples_leaf': 3,
 'forest_clf__n_estimators': 24}

In [24]:
# Ajustar el modelo con los nuevos parametros nombrandolo como  DT_Classifier_tunned
forest_clf_tunned  = RandomForestClassifier(max_depth = grid_RF.best_params_['forest_clf__max_depth'], 
                                            max_features = grid_RF.best_params_['forest_clf__max_features'],
                                            min_samples_leaf = grid_RF.best_params_['forest_clf__min_samples_leaf'],
                                            n_estimators = grid_RF.best_params_['forest_clf__n_estimators'],
                                            )

model_forest_clf_tunned = Pipeline(steps=[
    ('transformer', transformer),
    ('forest_clf_tunned', forest_clf_tunned)
])

In [25]:
#3. Ajustar el modelo
model_forest_clf_tunned.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16,
                                                   17, 18, 19, 20, 21, 22,
                                                   23])])),
                ('forest_clf_tunned',
                 RandomForestClassifier(max_depth=11, max_features='log2',
                                        min_samples_leaf=3, n_estimators=24))])

In [26]:
model_forest_clf_tunned.score(X_test, y_test)

0.8111111111111111

## Árbol de decisión

In [27]:
# Módulos del Modelo
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# modelo DecisionTreeClassifier llamado DT_Classifier
DT_Classifier  = DecisionTreeClassifier()

# Definiendo el modelo_DT con su pipeline
model_DT = Pipeline(  [
    ('transformer',transformer),
    ('DT_Classifier',DT_Classifier)
])

In [28]:
# Ajuste del modelo
model_DT.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16,
                                                   17, 18, 19, 20, 21, 22,
                                                   23])])),
                ('DT_Classifier', DecisionTreeClassifier())])

In [29]:
# Score
model_DT.score(X_test, y_test)

0.6777777777777778

In [30]:
# Definir los parametros con el diccionario params utilizando:
max_depth = [8, 10, 15, 20, 25, 30, 35]
min_samples_leaf = [10, 12, 14, 16, 18, 20]
params = {'DT_Classifier__max_depth':max_depth,
          'DT_Classifier__min_samples_leaf':min_samples_leaf
}

In [31]:
# Ajuste de los parametros max_depth y min_samples_leaf con  GridSearchCV
grid_DT = GridSearchCV(model_DT, param_grid = params, 
                       cv = 10,
                       scoring='neg_mean_squared_error',
                       n_jobs=-1,
                       verbose=2)

In [32]:
# Ajuste del modelo usando todas las posibles combinaciones de parametros 
grid_DT.fit(X, y)

Fitting 10 folds for each of 42 candidates, totalling 420 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4, 5,
                                                                          6, 7,
                                                                          8, 9,
                                                                          10,
                                                                          11,
                                                                          12,
                                                                          13,
                                                                          14,
                                                                          15,
                                                                          16,
                                                                          17,
                                                                          18,
                                                                          19,
                                                                          20,
                                                                          21,
                                                                          22,
                                                                          23])])),
                                       ('DT_Classifier',
                                        DecisionTreeClassifier())]),
             n_jobs=-1,
             param_grid={'DT_Classifier__max_depth': [8, 10, 15, 20, 25, 30,
                                                      35],
                         'DT_Classifier__min_samples_leaf': [10, 12, 14, 16, 18,
                                                             20]},
             scoring='neg_mean_squared_error', verbose=2)

In [33]:
#Encontrar los mejores parametros con best_params_
grid_DT.best_params_

{'DT_Classifier__max_depth': 15, 'DT_Classifier__min_samples_leaf': 18}

In [34]:
# Ajustar el modelo con los nuevos parametros nombrandolo como  DT_Classifier_tunned
DT_Classifier_tunned  = DecisionTreeClassifier(max_depth = grid_DT.best_params_['DT_Classifier__max_depth'], 
                                          min_samples_leaf = grid_DT.best_params_['DT_Classifier__min_samples_leaf'])

In [35]:
# Definiendo el modelo_DT con su pipeline
model_DT_tunned = Pipeline(  [
    ('transformer',transformer),
    ('DT_Classifier_tunned',DT_Classifier_tunned)
])

In [36]:
#3. Ajustar el modelo
model_DT_tunned.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16,
                                                   17, 18, 19, 20, 21, 22,
                                                   23])])),
                ('DT_Classifier_tunned',
                 DecisionTreeClassifier(max_depth=15, min_samples_leaf=18))])

In [37]:
#4 Score
model_DT_tunned.score(X_test, y_test)

0.6944444444444444

In [38]:
pd.DataFrame({
    'Modelo': ['DecisionTree', 'DecisionTree Tuned', 'RandomForest', 'RandomForest Tuned'],
    'Score': [model_DT.score(X_test, y_test), model_DT_tunned.score(X_test, y_test), 
                 model_forest_clf.score(X_test, y_test), model_forest_clf_tunned.score(X_test, y_test)]
})

,Modelo,Score
0,DecisionTree,0.677778
1,DecisionTree Tuned,0.694444
2,RandomForest,0.783333
3,RandomForest Tuned,0.811111


# Validando con el mejor modelo

In [39]:
model_forest_clf_tunned.score(X_validation, y_validation)

0.79